In [1]:
# Test 1: Basic workflow - Create and build a system incrementally
from netket_jsons import NetKetJSONManager
from netket_schemas import LatticeSchema, HilbertSpaceSchema, HamiltonianSchema

def test_basic_workflow():
    print("=== Test 1: Basic Workflow ===")
    
    # Initialize manager
    manager = NetKetJSONManager()
    
    # Create a new system
    system_id = manager.create_system("Heisenberg chain with 4 spins")
    print(f"✓ Created system: {system_id}")
    
    # Add lattice component
    manager.update_component("lattice", "1D chain with 4 sites")
    print("✓ Added lattice")
    
    # Add hilbert space
    manager.update_component("hilbert", "4 spin-1/2 particles")
    print("✓ Added hilbert space")
    
    # Add Hamiltonian
    manager.update_component("hamiltonian", "Heisenberg model with J=1.0")
    print("✓ Added Hamiltonian")
    
    # Check current system
    current_system = manager.systems[system_id]
    print(f"✓ System status: {current_system.status}")
    print(f"✓ Warnings: {len(current_system.warnings)}")
    print(f"✓ History length: {len(current_system.history)}")
    
    # Verify all components are set
    assert current_system.lattice is not None, "❌ Lattice not set"
    assert current_system.hilbert is not None, "❌ Hilbert space not set"
    assert current_system.hamiltonian is not None, "❌ Hamiltonian not set"
    print("✓ All components successfully set")
    
    return system_id, manager

system_id, manager = test_basic_workflow()

=== Test 1: Basic Workflow ===
✓ Created system: system_bf53dee3
✓ Added lattice
✓ Added hilbert space
✓ Added Hamiltonian
✓ System status: complete
✓ Warnings: 0
✓ History length: 3
✓ All components successfully set


In [2]:
# Test 2: Multiple systems with different configurations
def test_multiple_systems():
    print("\n=== Test 2: Multiple Systems ===")
    
    # Create a second system
    system_id2 = manager.create_system("2D Hubbard model")
    print(f"✓ Created second system: {system_id2}")
    
    # Build it step by step
    manager.update_component("lattice", "2D square lattice 3x3", system_id2)
    manager.update_component("hilbert", "9 fermions with spin", system_id2)
    manager.update_component("hamiltonian", "Hubbard model U=4.0, t=1.0", system_id2)
    print("✓ Built second system")
    
    # Create a third system
    system_id3 = manager.create_system("Kitaev honeycomb model")
    manager.update_component("lattice", "honeycomb lattice 2x2", system_id3)
    manager.update_component("hilbert", "8 spin-1/2 particles", system_id3)
    manager.update_component("hamiltonian", "Kitaev model Jx=1.0, Jy=1.0, Jz=1.0", system_id3)
    print("✓ Built third system")
    
    # Verify we have 3 systems
    assert len(manager.systems) == 3, f"❌ Expected 3 systems, got {len(manager.systems)}"
    print(f"✓ Total systems: {len(manager.systems)}")
    
    # List all systems
    systems_list = manager.list_systems()
    for sys_info in systems_list:
        print(f"  ✓ {sys_info['system_id']}: {sys_info['status']} - {sys_info['lattice']}")
    
    return system_id2, system_id3

system_id2, system_id3 = test_multiple_systems()


=== Test 2: Multiple Systems ===
✓ Created second system: system_b73c448e
✓ Built second system
✓ Built third system
✓ Total systems: 3
  ✓ system_bf53dee3: complete - chain of 4 sites
  ✓ system_b73c448e: complete - 3x3 square lattice
  ✓ system_87c3165b: complete - 2x2 honeycomb lattice


In [3]:
# Test 3: Update existing systems
def test_system_updates():
    print("\n=== Test 3: System Updates ===")
    
    # Modify the first system
    original_history_length = len(manager.systems[system_id].history)
    manager.update_component("hamiltonian", "XXZ model with J=1.0, Delta=0.5", system_id)
    print("✓ Updated Hamiltonian")
    
    # Check the history increased
    new_history_length = len(manager.systems[system_id].history)
    assert new_history_length > original_history_length, "❌ History not updated"
    print(f"✓ History updated: {original_history_length} → {new_history_length}")
    
    # Check the last history entry
    system = manager.systems[system_id]
    last_entry = system.history[-1]
    assert last_entry['action'] == 'set_hamiltonian', f"❌ Wrong action: {last_entry['action']}"
    print(f"✓ Last action: {last_entry['action']}")
    
    return system

updated_system = test_system_updates()


=== Test 3: System Updates ===
✓ Updated Hamiltonian
✓ History updated: 3 → 4
✓ Last action: set_hamiltonian


In [4]:
# Test 4: Save and reload systems
def test_persistence():
    print("\n=== Test 4: Persistence ===")
    
    # Save all systems
    for sys_id in manager.systems.keys():
        manager.save_system(sys_id)
        print(f"✓ Saved system {sys_id}")
    
    # Create a new manager instance (simulates restart)
    manager2 = NetKetJSONManager()
    expected_count = len(manager.systems)
    actual_count = len(manager2.systems)
    assert actual_count == expected_count, f"❌ Expected {expected_count} systems, got {actual_count}"
    print(f"✓ Loaded {actual_count} existing systems")
    
    # Load a specific system
    loaded_system = manager2.load_system(system_id)
    assert loaded_system.system_id == system_id, f"❌ Wrong system loaded: {loaded_system.system_id}"
    print(f"✓ Loaded system {system_id}")
    
    # Verify data integrity
    original_lattice = manager.systems[system_id].lattice.text
    loaded_lattice = loaded_system.lattice.text
    assert original_lattice == loaded_lattice, f"❌ Data corruption: {original_lattice} != {loaded_lattice}"
    print(f"✓ Data integrity verified: {loaded_lattice}")
    
    return manager2

manager2 = test_persistence()


=== Test 4: Persistence ===
✓ Saved system system_bf53dee3
✓ Saved system system_b73c448e
✓ Saved system system_87c3165b
✓ Loaded 3 existing systems
✓ Loaded system system_bf53dee3
✓ Data integrity verified: chain of 4 sites


In [ ]:
# Test 5: Text-based specifications with validation
def test_validation():
    print("\n=== Test 5: Validation ===")
    
    # Create a system with potential issues
    system_id4 = manager.create_system("Test validation")
    print(f"✓ Created validation test system: {system_id4}")
    
    # This should trigger a warning (too many fermions for lattice size)
    manager.update_component("lattice", "1D chain with 2 sites", system_id4)
    manager.update_component("hilbert", "5 fermions with spin-1/2", system_id4)  # More fermions than sites
    
    system = manager.systems[system_id4]
    assert len(system.warnings) > 0, "❌ Expected warnings for invalid configuration"
    print(f"✓ Generated warnings: {len(system.warnings)}")
    for warning in system.warnings:
        print(f"  ⚠️  {warning}")
    
    # Fix the issue
    manager.update_component("hilbert", "2 fermions with spin-1/2", system_id4)
    system = manager.systems[system_id4]
    print(f"✓ After fix - Warnings: {len(system.warnings)}")
    
    return system_id4


test_validation()


=== Test 5: Validation ===
✓ Created validation test system: system_c777b921
✓ Generated warnings: 1
  ⚠️  Invalid: 5 fermions cannot fit on 2 sites
✓ After fix - Warnings: 1


'system_c777b921'

In [ ]:
# Debug validation issue
def debug_validation():
    print("\n=== Debug Validation ===")
    
    # Create a fresh manager
    debug_manager = NetKetJSONManager("debug_quantum_systems")
    
    # Create system
    system_id = debug_manager.create_system("Debug test")
    print(f"✓ Created system: {system_id}")
    
    # Add lattice
    debug_manager.update_component("lattice", "1D chain with 2 sites", system_id)
    system = debug_manager.systems[system_id]
    print(f"✓ Added lattice: {system.lattice.text}")
    print(f"  Lattice extent: {system.lattice.extent}")
    print(f"  Warnings: {len(system.warnings)}")
    
    # Add hilbert space (should trigger warning)
    debug_manager.update_component("hilbert", "5 fermions with spin-1/2", system_id)
    system = debug_manager.systems[system_id]
    print(f"✓ Added hilbert space: {system.hilbert.text}")
    print(f"  Number of particles: {system.hilbert.n_particles}")
    print(f"  Warnings: {len(system.warnings)}")
    
    # Debug: Check what's happening
    print(f"\n=== Debug Info ===")
    print(f"Lattice extent: {system.lattice.extent}")
    print(f"Number of particles: {system.hilbert.n_particles}")
    print(f"Total sites: {system.lattice.extent[0] if system.lattice.extent else 'None'}")
    print(f"Should warn: {system.hilbert.n_particles > system.lattice.extent[0] if system.lattice.extent and system.hilbert.n_particles else 'N/A'}")
    
    # Manual validation check
    if system.lattice.extent and system.hilbert.n_particles:
        total_sites = system.lattice.extent[0]  # 1D chain
        if system.hilbert.n_particles > total_sites:
            print(f"⚠️ MANUAL CHECK: {system.hilbert.n_particles} > {total_sites} - should warn!")
        else:
            print(f"✓ MANUAL CHECK: {system.hilbert.n_particles} <= {total_sites} - no warning needed")
    
    # Show all warnings
    print(f"\n=== All Warnings ===")
    for i, warning in enumerate(system.warnings):
        print(f"  {i+1}. {warning}")
    
    return system_id

debug_validation()

debug_validation()


=== Debug Validation ===
✓ Created system: system_6e86c4fc
✓ Added lattice: chain of 2 sites
  Lattice extent: [2]
  Warnings: 0
✓ Added hilbert space: 5 fermions with spin-1/2
  Number of particles: 5
  Warnings: 1

=== Debug Info ===
Lattice extent: [2]
Number of particles: 5
Total sites: 2
Should warn: True
⚠️ MANUAL CHECK: 5 > 2 - should warn!

=== All Warnings ===
  1. Invalid: 5 fermions cannot fit on 2 sites

=== Debug Validation ===
✓ Created system: system_bbd0b439
✓ Added lattice: chain of 2 sites
  Lattice extent: [2]
  Warnings: 0
✓ Added hilbert space: 5 fermions with spin-1/2
  Number of particles: 5
  Warnings: 1

=== Debug Info ===
Lattice extent: [2]
Number of particles: 5
Total sites: 2
Should warn: True
⚠️ MANUAL CHECK: 5 > 2 - should warn!

=== All Warnings ===
  1. Invalid: 5 fermions cannot fit on 2 sites


'system_bbd0b439'

In [7]:
# Test 6: Complex system building with detailed specifications
def test_complex_specifications():
    print("\n=== Test 6: Complex Specifications ===")
    
    # Create a sophisticated system
    system_id5 = manager.create_system("Complex quantum system")
    print(f"✓ Created complex system: {system_id5}")
    
    # Detailed lattice specification
    lattice_spec = {
        "lattice_type": "triangular",
        "extent": [4, 4],
        "text": "4x4 triangular lattice"
    }
    manager.update_component("lattice", lattice_spec, system_id5)
    print("✓ Added detailed lattice spec")
    
    # Detailed hilbert space specification
    hilbert_spec = {
        "space_type": "fermion",
        "n_particles": 8,
        "spin": 0.5,
        "text": "8 fermions with spin-1/2"
    }
    manager.update_component("hilbert", hilbert_spec, system_id5)
    print("✓ Added detailed hilbert spec")
    
    # Complex Hamiltonian
    # Note: The schema infers model type and parameters from text
    hamiltonian_spec = {
        "text": "Heisenberg model with J=0.5 and hx=0.1"
    }
    manager.update_component("hamiltonian", hamiltonian_spec, system_id5)
    print("✓ Added detailed Hamiltonian spec")
    
    # Check the complete system
    system = manager.systems[system_id5]
    assert system.lattice is not None, "❌ Lattice not set"
    assert system.hilbert is not None, "❌ Hilbert space not set"
    assert system.hamiltonian is not None, "❌ Hamiltonian not set"
    print("✓ All complex components set")
    
    print(f"  Lattice: {system.lattice.text}")
    print(f"  Hilbert Space: {system.hilbert.text}")
    print(f"  Hamiltonian: {system.hamiltonian.text}")
    print(f"  Status: {system.status}")
    
    return system_id5

system_id5 = test_complex_specifications()


system_id5 = test_complex_specifications()


=== Test 6: Complex Specifications ===
✓ Created complex system: system_f99affc7
✓ Added detailed lattice spec
✓ Added detailed hilbert spec
✓ Added detailed Hamiltonian spec
✓ All complex components set
  Lattice: 4x4 triangular lattice
  Hilbert Space: 8 fermions with spin-1/2
  Hamiltonian: Heisenberg model with J=0.5, hx=0.1
  Status: complete

=== Test 6: Complex Specifications ===
✓ Created complex system: system_16b44b50
✓ Added detailed lattice spec
✓ Added detailed hilbert spec
✓ Added detailed Hamiltonian spec
✓ All complex components set
  Lattice: 4x4 triangular lattice
  Hilbert Space: 8 fermions with spin-1/2
  Hamiltonian: Heisenberg model with J=0.5, hx=0.1
  Status: complete


In [8]:
# Test 7: System comparison and analysis
def test_system_analysis():
    print("\n=== Test 7: System Analysis ===")
    
    # Compare different systems
    print("System comparison:")
    complete_systems = 0
    for sys_id, system in manager.systems.items():
        components = sum(1 for x in [system.lattice, system.hilbert, system.hamiltonian] if x is not None)
        if components == 3:
            complete_systems += 1
        
        print(f"  ✓ {sys_id}:")
        print(f"    Created: {system.created_at}")
        print(f"    Modified: {system.last_modified}")
        print(f"    Components: {components}/3")
        print(f"    History entries: {len(system.history)}")
        if system.warnings:
            print(f"    ⚠️  Warnings: {len(system.warnings)}")
    
    print(f"✓ Complete systems: {complete_systems}/{len(manager.systems)}")
    assert complete_systems > 0, "❌ No complete systems found"
    
    return complete_systems

complete_count = test_system_analysis()


=== Test 7: System Analysis ===
System comparison:
  ✓ system_bf53dee3:
    Created: 2025-06-19T18:47:28.048838
    Modified: 2025-06-19T18:47:28.112876
    Components: 3/3
    History entries: 4
  ✓ system_b73c448e:
    Created: 2025-06-19T18:47:28.084530
    Modified: 2025-06-19T18:47:28.090730
    Components: 3/3
    History entries: 3
    ⚠️  Warnings: 1
  ✓ system_87c3165b:
    Created: 2025-06-19T18:47:28.094644
    Modified: 2025-06-19T18:47:28.102092
    Components: 3/3
    History entries: 3
  ✓ system_c777b921:
    Created: 2025-06-19T18:47:28.159232
    Modified: 2025-06-19T18:47:28.174622
    Components: 2/3
    History entries: 3
    ⚠️  Warnings: 1
  ✓ system_f99affc7:
    Created: 2025-06-19T18:47:28.249595
    Modified: 2025-06-19T18:47:28.264669
    Components: 3/3
    History entries: 3
  ✓ system_16b44b50:
    Created: 2025-06-19T18:47:28.268948
    Modified: 2025-06-19T18:47:28.286329
    Components: 3/3
    History entries: 3
✓ Complete systems: 5/6


In [9]:
# Test 8: Export system data for analysis
def test_data_export():
    print("\n=== Test 8: Data Export ===")
    
    import json
    import os
    
    # Export all systems to a single file
    all_systems_data = {}
    for sys_id, system in manager.systems.items():
        all_systems_data[sys_id] = system.to_dict()
    
    export_file = "all_quantum_systems.json"
    with open(export_file, "w") as f:
        json.dump(all_systems_data, f, indent=2)
    
    assert os.path.exists(export_file), f"❌ Export file {export_file} not created"
    print(f"✓ Exported all systems to {export_file}")
    
    # Export specific system for sharing
    specific_system = manager.systems[system_id5]
    specific_file = f"system_{system_id5}_export.json"
    with open(specific_file, "w") as f:
        json.dump(specific_system.to_dict(), f, indent=2)
    
    assert os.path.exists(specific_file), f"❌ Specific export file {specific_file} not created"
    print(f"✓ Exported system {system_id5} to {specific_file}")
    
    # Verify export data integrity
    with open(specific_file, "r") as f:
        exported_data = json.load(f)
    
    assert exported_data["system_id"] == system_id5, "❌ Export data corruption"
    assert exported_data["lattice"] is not None, "❌ Lattice data missing in export"
    print("✓ Export data integrity verified")
    
    return export_file, specific_file

export_file, specific_file = test_data_export()


=== Test 8: Data Export ===
✓ Exported all systems to all_quantum_systems.json
✓ Exported system system_16b44b50 to system_system_16b44b50_export.json
✓ Export data integrity verified


In [10]:
# Test 9: Error handling and edge cases
def test_error_handling():
    print("\n=== Test 9: Error Handling ===")
    
    # Try to update non-existent system
    try:
        manager.update_component("lattice", "test", "non_existent_id")
        print("❌ Should have raised ValueError for non-existent system")
        assert False, "Expected error not raised"
    except ValueError as e:
        print(f"✓ Caught expected error: {e}")
    
    # Try to load non-existent file
    try:
        manager.load_system("non_existent_id")
        print("❌ Should have raised FileNotFoundError for non-existent file")
        assert False, "Expected error not raised"
    except FileNotFoundError as e:
        print(f"✓ Caught expected error: {e}")
    
    # Try invalid component type
    try:
        manager.update_component("invalid_type", "test")
        print("❌ Should have raised ValueError for invalid component type")
        assert False, "Expected error not raised"
    except ValueError as e:
        print(f"✓ Caught expected error: {e}")
    
    print("✓ All error handling tests passed")

test_error_handling()


=== Test 9: Error Handling ===
✓ Caught expected error: No such system to update.
✓ Caught expected error: System file quantum_systems/non_existent_id.json does not exist.
✓ Caught expected error: Unknown component type: invalid_type
✓ All error handling tests passed


In [11]:
# Test 10: Integration with NetKet (basic example)
def test_netket_integration():
    print("\n=== Test 10: NetKet Integration ===")
    
    try:
        import netket as nk
        
        # Get a complete system
        system = manager.systems[system_id]
        
        if system.lattice and system.hilbert and system.hamiltonian:  # Changed from particles to hilbert
            print("Building NetKet objects from schemas:")
            
            # Build lattice
            graph = system.lattice.to_netket_graph()  # Changed method name to match schema
            assert graph is not None, "❌ Failed to build NetKet graph"
            print(f"  ✓ Built graph: {type(graph)}")
            
            # Build Hilbert space
            hilbert = system.hilbert.to_netket_hilbert(graph)  # Changed method name and added graph parameter
            assert hilbert is not None, "❌ Failed to build NetKet Hilbert space"
            print(f"  ✓ Built Hilbert space: {type(hilbert)}")
            
            # Build Hamiltonian
            hamiltonian = system.hamiltonian.to_netket_hamiltonian(hilbert, graph)  # Changed method name and parameters
            assert hamiltonian is not None, "❌ Failed to build NetKet Hamiltonian"
            print(f"  ✓ Built Hamiltonian: {type(hamiltonian)}")
            
            # Store results in the system
            system.results["graph_type"] = str(type(graph))  # Changed from lattice_type to graph_type
            system.results["hilbert_type"] = str(type(hilbert))
            system.results["hamiltonian_type"] = str(type(hamiltonian))
            
            # Save the updated system
            manager.save_system(system_id)
            print("  ✓ Saved results to system")
            
            # Verify results were saved
            assert len(system.results) > 0, "❌ Results not stored"
            print(f"  ✓ Stored {len(system.results)} result entries")
            
        else:
            print("⚠️  Skipping NetKet integration - system not complete")
            
    except ImportError:
        print("⚠️  NetKet not available - skipping integration test")
    except Exception as e:
        print(f"❌ NetKet integration failed: {e}")

test_netket_integration()     


=== Test 10: NetKet Integration ===
Building NetKet objects from schemas:
  ✓ Built graph: <class 'netket.graph.lattice.Lattice'>
  ✓ Built Hilbert space: <class 'netket.hilbert.spin.Spin'>
  ✓ Built Hamiltonian: <class 'float'>
  ✓ Saved results to system
  ✓ Stored 3 result entries


In [12]:
# Final summary
def print_final_summary():
    print("\n" + "="*50)
    print("FINAL TEST SUMMARY")
    print("="*50)
    
    total_systems = len(manager.systems)
    complete_systems = sum(1 for sys in manager.systems.values() 
                          if all([sys.lattice, sys.hilbert, sys.hamiltonian]))  # Changed particles to hilbert
    
    print(f"✓ Total systems created: {total_systems}")
    print(f"✓ Complete systems: {complete_systems}")
    print(f"✓ Systems with warnings: {sum(1 for sys in manager.systems.values() if sys.warnings)}")
    print(f"✓ Total history entries: {sum(len(sys.history) for sys in manager.systems.values())}")
    
    # Check if quantum_systems directory exists and has files
    import os
    if os.path.exists("quantum_systems"):
        json_files = [f for f in os.listdir("quantum_systems") if f.endswith('.json')]
        print(f"✓ JSON files saved: {len(json_files)}")
    else:
        print("❌ quantum_systems directory not found")
    
    print("="*50)
    print("ALL TESTS COMPLETED SUCCESSFULLY! 🎉")
    print("="*50)

print_final_summary()


FINAL TEST SUMMARY
✓ Total systems created: 6
✓ Complete systems: 5
✓ Systems with warnings: 2
✓ Total history entries: 19
✓ JSON files saved: 6
ALL TESTS COMPLETED SUCCESSFULLY! 🎉
